In [63]:
import os
import pandas as pd

In [64]:
FILE = os.path.join(os.getcwd(), 'checked_list.csv')
df = pd.read_csv(FILE, encoding='utf-8-sig')
df['check'] = df[['check']].astype('Int64').fillna(0)

In [65]:
df['d'] = df['check'] - df['总数量']
df['insufficient'] = df['可用数量'] + df['d'] < 0
df['de-freeze'] = df[df['insufficient']]['可用数量'] + df[df['insufficient']]['d']
display(df[df['insufficient']])
display(df.dtypes)

,库位,料品,品名,批号,可用数量,冻结数量,总数量,check,d,insufficient,de-freeze
21,3A01,101201-15,马坝龙五星油粘米15,200817019,62,70,132,0,-132,True,-70
54,3A07,101301-75,太粮靓虾王香软米0.5*50,200813007,14,3,17,0,-17,True,-3
73,3A09,101301-75,太粮靓虾王香软米0.5*50,200708002,10,7,17,0,-17,True,-7
143,4A03,101301-10,太粮靓虾王香软米10,200816003,0,217,217,0,-217,True,-217
155,4A04,101401-25,(紫色)米皇坊小农粘米25,200805007,32,40,72,0,-72,True,-40
176,4A06,101301-08,太粮靓虾王香软米8,200808007,160,80,240,0,-240,True,-80
178,4A06,101306-10,（红香）靓虾王米10,200815006,2057,80,2137,0,-2137,True,-80
276,4AH4,101201-25,马坝龙五星油粘米25,200815005,13,40,53,0,-53,True,-40
288,4AH5,101306-25,（红香）靓虾王米25,200818003,170,30,200,0,-200,True,-30
302,4AP2,107001-32,宫粥东北大米2*10,200724012,19,7,26,0,-26,True,-7


库位              object
料品              object
品名              object
批号              object
可用数量             int64
冻结数量             int64
总数量              int64
check           object
d               object
insufficient      bool
de-freeze       object
dtype: object

In [77]:
tag = ['from', 'to', 'n']
checkio = pd.DataFrame(columns=tag)

for _, df_sku in df[df['d'] != 0].groupby(['料品', '批号']):
    if len(df_sku) == 1:
        pass
    else:
        move = list()
        # display(df_sku)
        ds = df_sku['d']
        m = - ds[ds < 0]   # minus
        p = ds[ds > 0]  # plus
        if len(m) > 0 and len(p) > 0:
            # display(- m, p)
            m.sort_values(ascending=False)
            p.sort_values(ascending=False)

            if m.sum() != p.sum():
                if m.sum() > p.sum():
                    more, less = m, p
                else:
                    more, less = p, m
                difference = more.sum() - less.sum()
                while difference >= more.iloc[0]:
                    difference -= more.pop(more.index[0])
                if difference > 0:
                    i = 0
                    while i < len(more) and difference <= more.iloc[i]:
                        i += 1
                    if more.iloc[i - 1] == 0:
                        more.pop(more.index[i - 1])
                    else:
                        more.iloc[i - 1] -= difference                
                more.sort_values(ascending=False)
            # display(- m, p)
            
            i = 0
            while i < len(m):
                n = m.iloc[i]
                matched = p[p == n].index
                if len(matched) > 0:  
                    move.append((m.index[i], matched[0], n))
                    m.pop(m.index[i])
                    p.pop(matched[0])
                else:
                    i += 1
            while len(m) > 0 and len(p) > 0:      
                move.append((m.index[0], p.index[0], min(m.iloc[0], p.iloc[0])))
                if m.iloc[0] > p.iloc[0]:
                    more, less, s_lt_d = m, p, True
                else:
                    more, less, s_lt_d = p, m, False
                more.iloc[0] -= less.pop(less.index[0])
                n = more.iloc[0]
                matched = less[less == n].index
                if len(matched) > 0:
                    if s_lt_d:     # supply larger than demand
                        move.append((more.index[0], matched[0], n))
                    else:
                        move.append((matched[0], more.index[0], n))                        
                    more.pop(more.index[0])
                    less.pop(matched[0])

            # display(move)  
            checkio = pd.concat([checkio, pd.DataFrame(move, columns=tag)], ignore_index=True)
checkio['sku'] = checkio['from'].map(lambda x: df['料品'][x])
checkio['batch'] = checkio['from'].map(lambda x: df['批号'][x])
checkio.set_index(['sku', 'batch'])
display(checkio)
assert len(checkio[checkio['n'] <= 0]) == 0
    

,from,to,n,sku,batch
0,285,411,12,100101-25,200817005
1,105,18,5,100101-40,200101001
2,45,32,32,100501-35,200814006
3,60,32,158,100501-35,200814006
4,34,52,91,100607-35,200817001
5,50,20,52,101101-32,200612007
6,50,297,31,101101-32,200612007
7,382,413,20,101202-10,200818007
8,339,309,2476,101301-10,200816003
9,301,298,8,101301-32,200608040
